In [ ]:
# to add the spiking model codebase to the path
import sys
sys.path.append('..')

%matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.animation import FuncAnimation
# plt.style.use('seaborn-pastel')

from brian2 import pA, Hz, second, ms
import numpy as np

from results import Results
from plot import (MidpointNormalize, weight_matrix, remove_axes_less, prune_figure, prune_figure_less, prune_figure_more,
                  plot_spike_raster, plot_firing_rate, plot_voltage_detail)

from generate import weight_matrix_from_flat_inds_weights, adjacency_matrix_from_flat_inds, generate_adjacency_indices_within
import pandas as pd
from sklearn.neighbors import KernelDensity
def cv(a):
    return a.std() / a.mean()

In [2]:
plt.close('all')

In [3]:
targetPath = 'C:/Users/mikejseay/Documents/BrianResults/'
# targetPath = 'C:/Users/mikejseay/Documents/BrianResultsHoff/'

In [4]:
targetFile1 = 'classicJercog_2000_0p25_cross-homeo-pre-scalar-homeo_guessGoodWeights2e3p025LogNormal_movAvg1_2021-08-05-11-02_results'  # converges at 330+
targetFile2 = 'classicJercog_2000_1_cross-homeo-pre-scalar-homeo_guessGoodWeights2e3p1LogNormal_fullConn_2021-08-13-09-38_results'  # converges at 230+

In [5]:
R1 = Results()
R1.init_from_file(targetFile1, targetPath)

if R1.p['useRule'][:5] == 'cross':
    R1.p['alpha2'] = -1e-12
    R1.p['tauPlasticityTrials'] = None
elif R1.p['useRule'] == 'homeo':
    R1.p['alpha2'] = -1e-12
    R1.p['tauPlasticityTrials'] = None

if 'wEEScale' not in R1.p:
    R1.p['wEEScale'] = None
    R1.p['wIEScale'] = None
    R1.p['wEIScale'] = None
    R1.p['wIIScale'] = None

importantInfoString = 'Name: {}\nEE: {}, IE: {}, EI: {}, II: {}\n tauFR={}, a1={:.4f} pA, a2={:.4f} pA, tauP={}'.format(R1.rID,
    R1.p['wEEScale'],R1.p['wIEScale'],R1.p['wEIScale'],R1.p['wIIScale'],
    R1.p['tauUpFRTrials'], R1.p['alpha1'] / pA, R1.p['alpha2'] / pA, R1.p['tauPlasticityTrials'])

wEEInit1 = weight_matrix_from_flat_inds_weights(R1.p['nExc'], R1.p['nExc'], R1.preEE, R1.posEE, R1.wEE_init)
wIEInit1 = weight_matrix_from_flat_inds_weights(R1.p['nExc'], R1.p['nInh'], R1.preIE, R1.posIE, R1.wIE_init)
wEIInit1 = weight_matrix_from_flat_inds_weights(R1.p['nInh'], R1.p['nExc'], R1.preEI, R1.posEI, R1.wEI_init)
wIIInit1 = weight_matrix_from_flat_inds_weights(R1.p['nInh'], R1.p['nInh'], R1.preII, R1.posII, R1.wII_init)

wEEFinal1 = weight_matrix_from_flat_inds_weights(R1.p['nExc'], R1.p['nExc'], R1.preEE, R1.posEE, R1.wEE_final)
wIEFinal1 = weight_matrix_from_flat_inds_weights(R1.p['nExc'], R1.p['nInh'], R1.preIE, R1.posIE, R1.wIE_final)
wEIFinal1 = weight_matrix_from_flat_inds_weights(R1.p['nInh'], R1.p['nExc'], R1.preEI, R1.posEI, R1.wEI_final)
wIIFinal1 = weight_matrix_from_flat_inds_weights(R1.p['nInh'], R1.p['nInh'], R1.preII, R1.posII, R1.wII_final)

wFullInit1 = np.block([[wEEInit1, wIEInit1], [-wEIInit1, -wIIInit1]])
wFullFinal1 = np.block([[wEEFinal1, wIEFinal1], [-wEIFinal1, -wIIFinal1]])

# get summed presynaptic input to each unit

wEEInitP1 = np.nansum(wEEInit1, 0)
wIEInitP1 = np.nansum(wIEInit1, 0)
wEIInitP1 = np.nansum(wEIInit1, 0)
wIIInitP1 = np.nansum(wIIInit1, 0)

wEEFinalP1 = np.nansum(wEEFinal1, 0)
wIEFinalP1 = np.nansum(wIEFinal1, 0)
wEIFinalP1 = np.nansum(wEIFinal1, 0)
wIIFinalP1 = np.nansum(wIIFinal1, 0)

print('wEE mult:', R1.wEE_final.mean() / R1.wEE_init.mean())
print('wIE mult:', R1.wIE_final.mean() / R1.wIE_init.mean())
print('wEI mult:', R1.wEI_final.mean() / R1.wEI_init.mean())
print('wII mult:', R1.wII_final.mean() / R1.wII_init.mean())

wEE mult: 0.9607961654663084
wIE mult: 0.9257208082411026
wEI mult: 1.046995162963867
wII mult: 0.8950419249358


In [6]:
R2 = Results()
R2.init_from_file(targetFile2, targetPath)

if R2.p['useRule'][:5] == 'cross':
    R2.p['alpha2'] = -1e-12
    R2.p['tauPlasticityTrials'] = None
elif R2.p['useRule'] == 'homeo':
    R2.p['alpha2'] = -1e-12
    R2.p['tauPlasticityTrials'] = None

if 'wEEScale' not in R2.p:
    R2.p['wEEScale'] = None
    R2.p['wIEScale'] = None
    R2.p['wEIScale'] = None
    R2.p['wIIScale'] = None

importantInfoString = 'Name: {}\nEE: {}, IE: {}, EI: {}, II: {}\n tauFR={}, a2={:.4f} pA, a2={:.4f} pA, tauP={}'.format(R2.rID,
    R2.p['wEEScale'],R2.p['wIEScale'],R2.p['wEIScale'],R2.p['wIIScale'],
    R2.p['tauUpFRTrials'], R2.p['alpha2'] / pA, R2.p['alpha2'] / pA, R2.p['tauPlasticityTrials'])

wEEInit2 = weight_matrix_from_flat_inds_weights(R2.p['nExc'], R2.p['nExc'], R2.preEE, R2.posEE, R2.wEE_init)
wIEInit2 = weight_matrix_from_flat_inds_weights(R2.p['nExc'], R2.p['nInh'], R2.preIE, R2.posIE, R2.wIE_init)
wEIInit2 = weight_matrix_from_flat_inds_weights(R2.p['nInh'], R2.p['nExc'], R2.preEI, R2.posEI, R2.wEI_init)
wIIInit2 = weight_matrix_from_flat_inds_weights(R2.p['nInh'], R2.p['nInh'], R2.preII, R2.posII, R2.wII_init)

wEEFinal2 = weight_matrix_from_flat_inds_weights(R2.p['nExc'], R2.p['nExc'], R2.preEE, R2.posEE, R2.wEE_final)
wIEFinal2 = weight_matrix_from_flat_inds_weights(R2.p['nExc'], R2.p['nInh'], R2.preIE, R2.posIE, R2.wIE_final)
wEIFinal2 = weight_matrix_from_flat_inds_weights(R2.p['nInh'], R2.p['nExc'], R2.preEI, R2.posEI, R2.wEI_final)
wIIFinal2 = weight_matrix_from_flat_inds_weights(R2.p['nInh'], R2.p['nInh'], R2.preII, R2.posII, R2.wII_final)

wFullInit2 = np.block([[wEEInit2, wIEInit2], [-wEIInit2, -wIIInit2]])
wFullFinal2 = np.block([[wEEFinal2, wIEFinal2], [-wEIFinal2, -wIIFinal2]])

# get summed presynaptic input to each unit

wEEInitP2 = np.nansum(wEEInit2, 0)
wIEInitP2 = np.nansum(wIEInit2, 0)
wEIInitP2 = np.nansum(wEIInit2, 0)
wIIInitP2 = np.nansum(wIIInit2, 0)

wEEFinalP2 = np.nansum(wEEFinal2, 0)
wIEFinalP2 = np.nansum(wIEFinal2, 0)
wEIFinalP2 = np.nansum(wEIFinal2, 0)
wIIFinalP2 = np.nansum(wIIFinal2, 0)

print('wEE mult:', R2.wEE_final.mean() / R2.wEE_init.mean())
print('wIE mult:', R2.wIE_final.mean() / R2.wIE_init.mean())
print('wEI mult:', R2.wEI_final.mean() / R2.wEI_init.mean())
print('wII mult:', R2.wII_final.mean() / R2.wII_init.mean())

wEE mult: 1.0086468378702806
wIE mult: 1.0044846428765195
wEI mult: 0.987797370323768
wII mult: 0.9804810418023001


In [7]:
# let's set up an artificial learning scenario
# we will start with weight matrices from the "full connectivity"
# we will also start with some post-convergence trial position
# we will take the individual unit firing rates at that position
# then we will calculate the proposed weight changes given each interpretation of the matrices

In [8]:
p = R1.p

In [9]:
plt.close('all')

In [230]:
# save the difference in the mean between the full and sparse II mat for each trial

dwIIFullMeanTrial1 = np.empty(R1.trialUpFRExc.size)
dwIISparseMeanTrial1 = np.empty(R1.trialUpFRExc.size)
dwIIFullMeanTrial2 = np.empty(R1.trialUpFRExc.size)
dwIISparseMeanTrial2 = np.empty(R1.trialUpFRExc.size)
dwIIFullMeanTrial3 = np.empty(R1.trialUpFRExc.size)
dwIISparseMeanTrial3 = np.empty(R1.trialUpFRExc.size)

commonRand = np.random.normal(0, 0.05, R1.trialUpFRExc.size)
dummyInd = 0

for trialInd in range(R1.trialUpFRExc.size):
    movAvgUpFRInhUnits1 = R1.trialUpFRInhUnits[trialInd, :] * Hz
    movAvgUpFRInhUnits2 = np.random.normal(1, 0.1, p['nInh']) * p['setUpFRInh']
    movAvgUpFRInhUnits3 = p['setUpFRInh'] + (commonRand[dummyInd] * p['setUpFRInh']) + np.random.normal(0, 0.1, p['nInh']) * p['setUpFRInh']
    dwIIHCurrent1 = -np.outer(movAvgUpFRInhUnits1, (p['setUpFRInh'] - movAvgUpFRInhUnits1))
    dwIIHCurrent2 = -np.outer(movAvgUpFRInhUnits2, (p['setUpFRInh'] - movAvgUpFRInhUnits2))
    dwIIHCurrent3 = -np.outer(movAvgUpFRInhUnits3, (p['setUpFRInh'] - movAvgUpFRInhUnits3))
    dwIIFullMeanTrial1[trialInd] = dwIIHCurrent1.mean()
    dwIISparseMeanTrial1[trialInd] = dwIIHCurrent1[R1.preII, R1.posII].mean()
    dwIIFullMeanTrial2[trialInd] = dwIIHCurrent2.mean()
    dwIISparseMeanTrial2[trialInd] = dwIIHCurrent2[R1.preII, R1.posII].mean()
    dwIIFullMeanTrial3[trialInd] = dwIIHCurrent3.mean()
    dwIISparseMeanTrial3[trialInd] = dwIIHCurrent3[R1.preII, R1.posII].mean()
    dummyInd += 1

dwIIFullSparseDiffTrials1 = dwIISparseMeanTrial1 - dwIIFullMeanTrial1
dwIIFullSparseDiffTrials2 = dwIISparseMeanTrial2 - dwIIFullMeanTrial2
dwIIFullSparseDiffTrials3 = dwIISparseMeanTrial3 - dwIIFullMeanTrial3

In [248]:
inhDF = pd.DataFrame({'fromSim': movAvgUpFRInhUnits1,
                      'randn1': movAvgUpFRInhUnits2,
                      'randn2': movAvgUpFRInhUnits3})

In [249]:
inhDF.describe()

,fromSim,randn1,randn2
count,400.000000,400.000000,400.000000
mean,14.033333,13.947064,12.998308
std,2.987182,1.411761,1.348215
min,5.925926,8.821659,9.516073
25%,11.851852,13.082204,12.034283
50%,14.074074,13.970907,13.023757
75%,16.296296,14.817289,13.894882
max,27.407407,19.184721,17.934174


In [239]:
movAvgUpFRInhUnits1

array([10.96835161, 13.96419389, 15.50741916, 15.03159535, 12.31492611,
       16.50240772, 16.54009926, 12.15876232, 13.68744189, 11.92884885,
       14.30919415, 14.54267131, 13.37427555, 13.10935694, 15.36449115,
       14.74963517, 11.92286259, 13.14465449, 16.16563986, 14.87173395,
       14.00152169, 11.40746715, 12.82492601, 14.85235469, 17.64273595,
       12.0991261 , 15.63406829, 13.52204378, 13.31876266, 11.97970111,
       14.14627636, 13.34202367, 14.11309833, 11.49987638, 12.20961378,
       13.98218952, 15.45860717, 13.83910486, 12.99186937, 14.27815338,
       13.00489766, 13.8382195 , 15.90129542, 13.98464609, 14.14552628,
       13.65384396, 15.02089114, 15.45002763, 14.0585729 , 14.34064794,
       13.82542753, 17.67477959, 13.72575322, 14.01202861, 14.37145556,
       12.95237658, 13.91304567, 14.80426535, 14.32902926, 13.27927077,
       14.92032034, 13.27755192, 14.54643943, 15.02017853, 13.98723004,
       14.16963363, 12.61355437, 14.42760286, 15.14589581, 16.36

In [241]:
movAvgUpFRInhUnits3

array([14.16245493, 13.62164697, 13.2122935 , 12.58541912, 13.66689132,
       13.43367059, 10.69679925, 14.34147416, 14.30662458, 10.06583546,
       13.13784935, 13.77718692, 11.69692626,  9.86202219, 11.39016976,
       13.75049602, 10.70925943, 12.57482428, 13.57225973, 12.69888773,
       12.07545126, 12.79017838, 13.94212027, 12.48593377, 14.92542833,
       11.81330545, 12.32251529, 11.64909078, 11.8220928 , 11.38912547,
       11.97599388, 13.86985063, 13.6236914 , 13.26141735, 12.32282737,
       17.38392939, 11.71029941, 14.27376873, 13.91437907, 13.65947965,
       11.17239982, 11.0380421 , 15.14723432, 12.70006046, 13.92422354,
       11.96992553, 12.11426149, 10.92477905, 12.90382266, 11.21494855,
       13.9436733 , 12.51178994, 12.83462717, 13.40860581, 11.76141646,
       11.81783762, 13.48426564, 13.44506673, 14.21224583, 14.60322133,
       12.67497904, 11.95280266, 13.17542213, 14.36505828, 12.92917941,
       10.38235973, 11.4970668 , 13.94542835, 10.37085329, 13.43

In [237]:
f, ax = plt.subplots()
ax.hlines(0, 0, R1.trialUpFRExc.size, linestyles='--')
ax.plot(dwIIFullMeanTrial1, alpha=0.5, color='b')
ax.plot(dwIISparseMeanTrial1, alpha=0.5, color='g')
ax.hlines(dwIIFullSparseDiffTrials1.mean(), 0, R1.trialUpFRExc.size)

In [235]:
f, ax = plt.subplots()
ax.hlines(0, 0, R1.trialUpFRExc.size, linestyles='--')
ax.plot(dwIIFullMeanTrial2, alpha=0.5, color='b')
ax.plot(dwIISparseMeanTrial2, alpha=0.5, color='g')
ax.hlines(dwIIFullSparseDiffTrials2.mean(), 0, R1.trialUpFRExc.size)

In [236]:
f, ax = plt.subplots()
ax.hlines(0, 0, R1.trialUpFRExc.size, linestyles='--')
ax.plot(dwIIFullMeanTrial3, alpha=0.5, color='b')
ax.plot(dwIISparseMeanTrial3, alpha=0.5, color='g')
ax.hlines(dwIIFullSparseDiffTrials3.mean(), 0, R1.trialUpFRExc.size)

In [ ]:
# goal here is to actually calculate the dw on EVERY trial (massive array)
# to save memory we will accumulate a sum and divide at the end

startTrialInd = 400
endTrialInd = R1.trialUpFRExc.size

nTrials = endTrialInd - startTrialInd
nUnits = p['nInh']

dwEEHTrialSum = np.zeros((p['nExc'], p['nExc']))
dwEIHTrialSum = np.zeros((p['nInh'], p['nExc']))
dwIEHTrialSum = np.zeros((p['nExc'], p['nInh']))
dwIIHTrialSum = np.zeros((p['nInh'], p['nInh']))

trialDiffII = np.empty(())

for trialInd in range(startTrialInd, endTrialInd):
    movAvgUpFRExcUnits = R1.trialUpFRExcUnits[trialInd, :] * Hz
    movAvgUpFRInhUnits = R1.trialUpFRInhUnits[trialInd, :] * Hz
    dwEEHTrialSum += np.outer(movAvgUpFRExcUnits, (p['setUpFRExc'] - movAvgUpFRExcUnits))
    dwEIHTrialSum += -np.outer(movAvgUpFRInhUnits, (p['setUpFRExc'] - movAvgUpFRExcUnits)) 
    dwIEHTrialSum += np.outer(movAvgUpFRExcUnits, (p['setUpFRInh'] - movAvgUpFRInhUnits)) 
    dwIIHTrialSum += -np.outer(movAvgUpFRInhUnits, (p['setUpFRInh'] - movAvgUpFRInhUnits)) 

dwEEHHat = dwEEHTrialSum / nTrials
dwEIHHat = dwEIHTrialSum / nTrials
dwIEHHat = dwIEHTrialSum / nTrials
dwIIHHat = dwIIHTrialSum / nTrials

dwEEHHatS = np.full((p['nExc'], p['nExc']), np.nan)
dwEIHHatS = np.full((p['nInh'], p['nExc']), np.nan)
dwIEHHatS = np.full((p['nExc'], p['nInh']), np.nan)
dwIIHHatS = np.full((p['nInh'], p['nInh']), np.nan)

dwEEHHatS[R1.preEE, R1.posEE] = dwEEHHat[R1.preEE, R1.posEE]
dwEIHHatS[R1.preEI, R1.posEI] = dwEIHHat[R1.preEI, R1.posEI]
dwIEHHatS[R1.preIE, R1.posIE] = dwIEHHat[R1.preIE, R1.posIE]
dwIIHHatS[R1.preII, R1.posII] = dwIIHHat[R1.preII, R1.posII]

In [197]:
dwIIHHat.sum()

11035.048273874077

In [198]:
np.nansum(dwIIHHatS)

-8430.28191746489

In [10]:
print(np.nanmean(dwEEHHatS) - dwEEHHat.mean())
print(np.nanmean(dwEIHHatS) - dwEIHHat.mean())
print(np.nanmean(dwIEHHatS) - dwIEHHat.mean())
print(np.nanmean(dwIIHHatS) - dwIIHHat.mean())

In [26]:
# now let's repeat the same thing but draw the firing rates from a normal distribution centered around the setpoints
# and see if the bias is still there...

startTrialInd = 400
endTrialInd = R1.trialUpFRExc.size

nTrials = endTrialInd - startTrialInd
nUnits = p['nInh']

dwEEHTrialSumF = np.zeros((p['nExc'], p['nExc']))
dwEIHTrialSumF = np.zeros((p['nInh'], p['nExc']))
dwIEHTrialSumF = np.zeros((p['nExc'], p['nInh']))
dwIIHTrialSumF = np.zeros((p['nInh'], p['nInh']))

for trialInd in range(startTrialInd, endTrialInd):
    movAvgUpFRExcUnits = np.random.normal(1, 0.1, p['nExc']) * p['setUpFRExc']
    movAvgUpFRInhUnits = np.random.normal(1, 0.1, p['nInh']) * p['setUpFRInh']
    dwEEHTrialSumF += np.outer(movAvgUpFRExcUnits, (p['setUpFRExc'] - movAvgUpFRExcUnits))
    dwEIHTrialSumF += -np.outer(movAvgUpFRInhUnits, (p['setUpFRExc'] - movAvgUpFRExcUnits)) 
    dwIEHTrialSumF += np.outer(movAvgUpFRExcUnits, (p['setUpFRInh'] - movAvgUpFRInhUnits)) 
    dwIIHTrialSumF += -np.outer(movAvgUpFRInhUnits, (p['setUpFRInh'] - movAvgUpFRInhUnits)) 

dwEEHHatF = dwEEHTrialSumF / nTrials
dwEIHHatF = dwEIHTrialSumF / nTrials
dwIEHHatF = dwIEHTrialSumF / nTrials
dwIIHHatF = dwIIHTrialSumF / nTrials

dwEEHHatSF = np.full((p['nExc'], p['nExc']), np.nan)
dwEIHHatSF = np.full((p['nInh'], p['nExc']), np.nan)
dwIEHHatSF = np.full((p['nExc'], p['nInh']), np.nan)
dwIIHHatSF = np.full((p['nInh'], p['nInh']), np.nan)

dwEEHHatSF[R1.preEE, R1.posEE] = dwEEHHatF[R1.preEE, R1.posEE]
dwEIHHatSF[R1.preEI, R1.posEI] = dwEIHHatF[R1.preEI, R1.posEI]
dwIEHHatSF[R1.preIE, R1.posIE] = dwIEHHatF[R1.preIE, R1.posIE]
dwIIHHatSF[R1.preII, R1.posII] = dwIIHHatF[R1.preII, R1.posII]

In [134]:
f, ax = plt.subplots(1, 2)
ax[0].hist(R1.preII, bins=400)
ax[1].hist(R1.posII, bins=400)

(array([ 87.,  90.,  94.,  94., 106.,  99., 109., 103., 105.,  83.,  89.,
        102., 107., 101.,  90.,  95.,  97., 114., 103., 109.,  93.,  89.,
         97.,  91., 107., 102.,  91.,  81.,  95.,  99.,  86., 108.,  99.,
         95., 110.,  98., 114.,  92., 108., 103., 102., 106.,  96., 113.,
         91., 100.,  92., 103.,  86., 100., 109.,  82.,  89., 104., 100.,
         98.,  95., 108.,  94.,  99.,  99., 101., 108., 109., 107., 125.,
        118.,  96., 112.,  99.,  97.,  84., 102.,  93., 105.,  87.,  98.,
         94.,  87., 103., 109., 120.,  99.,  99., 100., 106.,  86., 111.,
        105., 107., 126., 113., 102.,  86.,  90., 101.,  97.,  95., 104.,
         88., 109., 112.,  95.,  93., 112., 106., 100., 108., 110., 100.,
         98., 108.,  99., 106.,  94.,  87.,  99.,  95., 101., 103.,  90.,
        104.,  91.,  99.,  98., 107., 110., 105.,  97.,  98.,  86., 117.,
         88., 104.,  92., 101.,  89., 109., 109.,  93.,  91., 105., 106.,
         94., 103.,  88., 106., 115., 

In [27]:
print(np.nanmean(dwEEHHatSF) - dwEEHHatF.mean())
print(np.nanmean(dwEIHHatSF) - dwEIHHatF.mean())
print(np.nanmean(dwIEHHatSF) - dwIEHHatF.mean())
print(np.nanmean(dwIIHHatSF) - dwIIHHatF.mean())

0.00015243796536301138
-0.00011669231842389213
0.00022475333045161427
-0.004139629986892811


In [51]:
commonRand = np.random.normal(0, 0.05, nTrials)
commonRand

array([ 0.0058333 , -0.04720438, -0.01898308, ...,  0.00893685,
        0.04759371,  0.013453  ])

In [52]:
commonRand.min()

-0.19739829506812778

In [58]:
test = np.random.normal(0, 0.015, p['nExc'])

In [59]:
test.min()

-0.04677020554953231

In [104]:
recordFakeExc = np.empty((nTrials, p['nExc']))
recordFakeInh = np.empty((nTrials, p['nInh']))

commonRand = np.random.normal(0, 0.05, nTrials)
dummyInd = 0
for trialInd in range(startTrialInd, endTrialInd):
    recordFakeExc[dummyInd, :] = p['setUpFRExc'] + (commonRand[dummyInd] * p['setUpFRExc']) + np.random.normal(0, 0.1, p['nExc']) * p['setUpFRExc']
    recordFakeInh[dummyInd, :] = p['setUpFRInh'] - (commonRand[dummyInd] * p['setUpFRInh']) + np.random.normal(0, 0.1, p['nInh']) * p['setUpFRInh']
    dummyInd += 1

In [105]:
f, ax = plt.subplots()
ax.scatter(recordFakeExc.mean(1), recordFakeInh.mean(1))

In [106]:
f, ax = plt.subplots()
ax.hist(recordFakeInh.ravel(), bins=40)

(array([1.00000e+00, 9.00000e+00, 1.90000e+01, 4.60000e+01, 1.61000e+02,
        4.08000e+02, 9.98000e+02, 2.09100e+03, 4.54100e+03, 8.80300e+03,
        1.67360e+04, 2.88310e+04, 4.73870e+04, 7.23350e+04, 1.05104e+05,
        1.41485e+05, 1.80235e+05, 2.14663e+05, 2.41483e+05, 2.53715e+05,
        2.51379e+05, 2.33036e+05, 2.02700e+05, 1.66383e+05, 1.28092e+05,
        9.21790e+04, 6.26950e+04, 3.98560e+04, 2.39740e+04, 1.32580e+04,
        7.05100e+03, 3.49200e+03, 1.65100e+03, 7.43000e+02, 2.94000e+02,
        1.09000e+02, 3.90000e+01, 1.30000e+01, 3.00000e+00, 2.00000e+00]),
 array([ 6.15632997,  6.55175446,  6.94717895,  7.34260345,  7.73802794,
         8.13345243,  8.52887693,  8.92430142,  9.31972592,  9.71515041,
        10.1105749 , 10.5059994 , 10.90142389, 11.29684839, 11.69227288,
        12.08769737, 12.48312187, 12.87854636, 13.27397085, 13.66939535,
        14.06481984, 14.46024434, 14.85566883, 15.25109332, 15.64651782,
        16.04194231, 16.43736681, 16.8327913 , 17

In [112]:
f, ax = plt.subplots()
ax.hist(R1.trialUpFRInhUnits[startTrialInd:endTrialInd].ravel(), bins=40)

(array([8.00000e+00, 3.70000e+01, 1.89000e+02, 5.71000e+02, 1.56100e+03,
        3.85100e+03, 2.45660e+04, 2.99630e+04, 4.78130e+04, 7.25390e+04,
        1.02231e+05, 1.34592e+05, 2.13403e+05, 3.53452e+05, 2.29660e+05,
        2.22919e+05, 2.11763e+05, 1.92944e+05, 2.00685e+05, 1.32222e+05,
        1.67432e+05, 6.72480e+04, 4.62630e+04, 3.21090e+04, 2.17150e+04,
        1.63570e+04, 1.20280e+04, 3.40400e+03, 2.05700e+03, 1.07900e+03,
        6.47000e+02, 3.98000e+02, 1.52000e+02, 1.04000e+02, 2.30000e+01,
        5.00000e+00, 4.00000e+00, 1.00000e+00, 2.00000e+00, 3.00000e+00]),
 array([ 0.73529412,  1.58728214,  2.43927015,  3.29125817,  4.14324619,
         4.9952342 ,  5.84722222,  6.69921024,  7.55119826,  8.40318627,
         9.25517429, 10.10716231, 10.95915033, 11.81113834, 12.66312636,
        13.51511438, 14.3671024 , 15.21909041, 16.07107843, 16.92306645,
        17.77505447, 18.62704248, 19.4790305 , 20.33101852, 21.18300654,
        22.03499455, 22.88698257, 23.73897059, 24

In [114]:
f, ax = plt.subplots()
weight_matrix(ax, R1.trialUpFRInhUnits[startTrialInd:endTrialInd].T)

In [116]:
f, ax = plt.subplots()
ax.plot(R1.trialUpFRInhUnits[startTrialInd:endTrialInd])

In [117]:
R1.trialUpFRInhUnits[startTrialInd:endTrialInd].shape

(6365, 400)

In [122]:
frCorrBetweenTrials = np.corrcoef(R1.trialUpFRInhUnits[startTrialInd:endTrialInd])
frCorrBetweenUnits = np.corrcoef(R1.trialUpFRInhUnits[startTrialInd:endTrialInd].T)

In [123]:
frCorrBetweenTrials[np.diag_indices_from(frCorrBetweenTrials)] = 0
frCorrBetweenUnits[np.diag_indices_from(frCorrBetweenUnits)] = 0

In [124]:
f, ax = plt.subplots(1, 2)
weight_matrix(ax[0], frCorrBetweenTrials)
weight_matrix(ax[1], frCorrBetweenUnits)

In [142]:
dwIIHHatNoDiag = dwIIHHat.copy()
dwIIHHatNoDiag[np.diag_indices_from(dwIIHHatNoDiag)] = np.nan

In [143]:
useMin = np.min([np.nanmin(dwIIHHatS), np.nanmin(dwIIHHatNoDiag),])
useMax = np.max([np.nanmax(dwIIHHatS), np.nanmax(dwIIHHatNoDiag),])

useBins = np.linspace(useMin, useMax, 100)

f, ax = plt.subplots()
ax.hist(dwIIHHatS[~np.isnan(dwIIHHatS)], bins=useBins, density=True, histtype='step', color='b')
ax.hist(dwIIHHatNoDiag[~np.isnan(dwIIHHatNoDiag)], bins=useBins, density=True, histtype='step', color='g')
ax.vlines(np.nanmean(dwIIHHatS), 0, ax.get_ylim()[1], colors='b', linestyle='--')
ax.vlines(np.nanmean(dwIIHHatNoDiag), 0, ax.get_ylim()[1], colors='g', linestyle='--')
ax.legend(('actual', 'all'))

In [208]:
dwIIHNoDiag = dwIIH.copy()
dwIIHNoDiag[np.diag_indices_from(dwIIHNoDiag)] = np.nan

In [214]:
useMin = np.min([np.nanmin(dwIIHS), np.nanmin(dwIIHNoDiag),])
useMax = np.max([np.nanmax(dwIIHS), np.nanmax(dwIIHNoDiag),])

useBins = np.linspace(useMin, useMax, 100)

f, ax = plt.subplots()
ax.hist(dwIIHS[~np.isnan(dwIIHS)], bins=useBins, density=True, histtype='step', color='b')
ax.hist(dwIIHNoDiag[~np.isnan(dwIIHNoDiag)], bins=useBins, density=True, histtype='step', color='g')
ax.vlines(np.nanmean(dwIIHS), 0, ax.get_ylim()[1], colors='b', linestyle='--')
ax.vlines(np.nanmean(dwIIHNoDiag), 0, ax.get_ylim()[1], colors='g', linestyle='--')
ax.legend(('actual', 'all'))

In [204]:
print(np.nanmean(dwIIHS))

-1.7381049


In [176]:
dwIIHHatSRandRavel = np.random.choice(dwIIHHat.ravel(), int(dwIIHHat.size / 4), replace=False)
dwIIHHatSRandRavel.mean()

0.07116120226433066

In [181]:
preInds, postInds = generate_adjacency_indices_within(p['nInh'], p['propConnect'], allowAutapses=False)
dwIIHHatSNewSyns = dwIIHHat[preInds, postInds]
dwIIHHatSNewSyns.mean()

0.03733229538960694

In [192]:
ravelIndsII = np.ravel_multi_index((R1.preII, R1.posII,), (p['nInh'], p['nInh']))

In [194]:
dwIIHHatSRavel = dwIIHHat.ravel()[ravelIndsII]
dwIIHHatSRavel.mean()

-0.21075704793662225

In [140]:
dwIIHHat.mean()

0.06896905171171298

In [141]:
np.nanmean(dwIIHHatNoDiag)

0.04081709116325066

In [130]:
np.nanmean(dwIIHHatS)

-0.21075704793662225

In [91]:
# now let's repeat the same thing as the fake, but the firing rates are correlated with each other

startTrialInd = 400
endTrialInd = R1.trialUpFRExc.size

nTrials = endTrialInd - startTrialInd
nUnits = p['nInh']

dwEEHTrialSumF2 = np.zeros((p['nExc'], p['nExc']))
dwEIHTrialSumF2 = np.zeros((p['nInh'], p['nExc']))
dwIEHTrialSumF2 = np.zeros((p['nExc'], p['nInh']))
dwIIHTrialSumF2 = np.zeros((p['nInh'], p['nInh']))

commonRand = np.random.normal(0, 0.05, nTrials)
dummyInd = 0
for trialInd in range(startTrialInd, endTrialInd):
    movAvgUpFRExcUnits = p['setUpFRExc'] + (commonRand[dummyInd] * p['setUpFRExc']) + np.random.normal(0, 0.1, p['nExc']) * p['setUpFRExc']
    movAvgUpFRInhUnits = p['setUpFRInh'] + (commonRand[dummyInd] * p['setUpFRInh']) + np.random.normal(0, 0.1, p['nInh']) * p['setUpFRInh']
    dwEEHTrialSumF2 += np.outer(movAvgUpFRExcUnits, (p['setUpFRExc'] - movAvgUpFRExcUnits))
    dwEIHTrialSumF2 += -np.outer(movAvgUpFRInhUnits, (p['setUpFRExc'] - movAvgUpFRExcUnits)) 
    dwIEHTrialSumF2 += np.outer(movAvgUpFRExcUnits, (p['setUpFRInh'] - movAvgUpFRInhUnits)) 
    dwIIHTrialSumF2 += -np.outer(movAvgUpFRInhUnits, (p['setUpFRInh'] - movAvgUpFRInhUnits))
    dummyInd += 1

dwEEHHatF2 = dwEEHTrialSumF2 / nTrials
dwEIHHatF2 = dwEIHTrialSumF2 / nTrials
dwIEHHatF2 = dwIEHTrialSumF2 / nTrials
dwIIHHatF2 = dwIIHTrialSumF2 / nTrials

dwEEHHatSF2 = np.full((p['nExc'], p['nExc']), np.nan)
dwEIHHatSF2 = np.full((p['nInh'], p['nExc']), np.nan)
dwIEHHatSF2 = np.full((p['nExc'], p['nInh']), np.nan)
dwIIHHatSF2 = np.full((p['nInh'], p['nInh']), np.nan)

dwEEHHatSF2[R1.preEE, R1.posEE] = dwEEHHatF2[R1.preEE, R1.posEE]
dwEIHHatSF2[R1.preEI, R1.posEI] = dwEIHHatF2[R1.preEI, R1.posEI]
dwIEHHatSF2[R1.preIE, R1.posIE] = dwIEHHatF2[R1.preIE, R1.posIE]
dwIIHHatSF2[R1.preII, R1.posII] = dwIIHHatF2[R1.preII, R1.posII]

In [92]:
print(np.nanmean(dwEEHHatSF2) - dwEEHHatF2.mean())
print(np.nanmean(dwEIHHatSF2) - dwEIHHatF2.mean())
print(np.nanmean(dwIEHHatSF2) - dwIEHHatF2.mean())
print(np.nanmean(dwIIHHatSF2) - dwIIHHatF2.mean())

0.00011800251606422507
-0.00010959104678856524
-0.0001191600556880168
-0.005490293082037856


In [100]:
# now let's repeat the same thing as the fake, but the firing rates are correlated with each other

startTrialInd = 400
endTrialInd = R1.trialUpFRExc.size

nTrials = endTrialInd - startTrialInd
nUnits = p['nInh']

dwEEHTrialSumF3 = np.zeros((p['nExc'], p['nExc']))
dwEIHTrialSumF3 = np.zeros((p['nInh'], p['nExc']))
dwIEHTrialSumF3 = np.zeros((p['nExc'], p['nInh']))
dwIIHTrialSumF3 = np.zeros((p['nInh'], p['nInh']))

commonRand = np.random.normal(0, 0.05, nTrials)
dummyInd = 0
for trialInd in range(startTrialInd, endTrialInd):
    movAvgUpFRExcUnits = p['setUpFRExc'] + (commonRand[dummyInd] * p['setUpFRExc']) + np.random.normal(0, 0.1, p['nExc']) * p['setUpFRExc']
    movAvgUpFRInhUnits = p['setUpFRInh'] - (commonRand[dummyInd] * p['setUpFRInh']) + np.random.normal(0, 0.1, p['nInh']) * p['setUpFRInh']
    dwEEHTrialSumF3 += np.outer(movAvgUpFRExcUnits, (p['setUpFRExc'] - movAvgUpFRExcUnits))
    dwEIHTrialSumF3 += -np.outer(movAvgUpFRInhUnits, (p['setUpFRExc'] - movAvgUpFRExcUnits)) 
    dwIEHTrialSumF3 += np.outer(movAvgUpFRExcUnits, (p['setUpFRInh'] - movAvgUpFRInhUnits)) 
    dwIIHTrialSumF3 += -np.outer(movAvgUpFRInhUnits, (p['setUpFRInh'] - movAvgUpFRInhUnits))
    dummyInd += 1

dwEEHHatF3 = dwEEHTrialSumF3 / nTrials
dwEIHHatF3 = dwEIHTrialSumF3 / nTrials
dwIEHHatF3 = dwIEHTrialSumF3 / nTrials
dwIIHHatF3 = dwIIHTrialSumF3 / nTrials

dwEEHHatSF3 = np.full((p['nExc'], p['nExc']), np.nan)
dwEIHHatSF3 = np.full((p['nInh'], p['nExc']), np.nan)
dwIEHHatSF3 = np.full((p['nExc'], p['nInh']), np.nan)
dwIIHHatSF3 = np.full((p['nInh'], p['nInh']), np.nan)

dwEEHHatSF3[R1.preEE, R1.posEE] = dwEEHHatF3[R1.preEE, R1.posEE]
dwEIHHatSF3[R1.preEI, R1.posEI] = dwEIHHatF3[R1.preEI, R1.posEI]
dwIEHHatSF3[R1.preIE, R1.posIE] = dwIEHHatF3[R1.preIE, R1.posIE]
dwIIHHatSF3[R1.preII, R1.posII] = dwIIHHatF3[R1.preII, R1.posII]

In [101]:
print(np.nanmean(dwEEHHatSF3) - dwEEHHatF3.mean())
print(np.nanmean(dwEIHHatSF3) - dwEIHHatF3.mean())
print(np.nanmean(dwIEHHatSF3) - dwIEHHatF3.mean())
print(np.nanmean(dwIIHHatSF3) - dwIIHHatF3.mean())

0.00019182456339893184
0.00012037069305304016
0.00013306494063830554
-0.0062561915408352875


In [93]:
dwIIHHatNorm = dwIIHHat.copy()
dwIIHHatNorm[np.diag_indices_from(dwIIHHatNorm)] = 0
dwIIHHatNorm = dwIIHHatNorm / np.max(np.fabs(dwIIHHatNorm))

In [94]:
dwIIHHatNormF = dwIIHHatF.copy()
dwIIHHatNormF[np.diag_indices_from(dwIIHHatNormF)] = 0
dwIIHHatNormF = dwIIHHatNormF / np.max(np.fabs(dwIIHHatNormF))

In [95]:
dwIIHHatNormF2 = dwIIHHatF2.copy()
dwIIHHatNormF2[np.diag_indices_from(dwIIHHatNormF2)] = 0
dwIIHHatNormF2 = dwIIHHatNormF2 / np.max(np.fabs(dwIIHHatNormF2))

In [102]:
dwIIHHatNormF3 = dwIIHHatF3.copy()
dwIIHHatNormF3[np.diag_indices_from(dwIIHHatNormF3)] = 0
dwIIHHatNormF3 = dwIIHHatNormF3 / np.max(np.fabs(dwIIHHatNormF3))

In [84]:
dwIIHHatNorm[:, 0]

array([ 0.00000000e+00,  3.30692436e-01,  5.85084199e-02, -3.25452263e-01,
       -1.10724434e-01, -9.33513766e-02,  2.93308681e-03,  1.40905829e-01,
        9.64936185e-02, -4.36818873e-02,  5.92230430e-02,  2.33087621e-01,
       -1.84758889e-01, -2.56458176e-01,  8.24481270e-02,  8.75811424e-02,
        5.34978521e-02, -1.50041197e-02, -4.15388107e-02,  9.48994939e-03,
        1.78297394e-01,  2.28724494e-01,  1.33324278e-01,  5.31235363e-02,
       -1.16529839e-01,  1.87293830e-01, -5.40561901e-02, -1.85866425e-01,
        6.77133913e-02,  5.32678735e-02,  2.33581667e-01,  3.97803748e-02,
       -1.59883331e-01,  3.46935795e-02,  2.20919580e-01, -2.84875679e-01,
       -1.29707955e-01,  5.52200177e-02, -8.67990751e-02, -9.48860949e-02,
        2.53980659e-01,  2.23057746e-01,  3.03130502e-01,  1.25172789e-02,
        4.58420660e-02,  2.79070225e-01,  3.58517338e-02, -2.33090827e-02,
       -1.04933439e-01,  1.65360077e-01, -8.76888034e-02, -2.52084854e-03,
        1.16524275e-01, -

In [85]:
dwIIHHatNormF[:, 0]

array([ 0.        , -0.11919992, -0.12740031, -0.18212857, -0.11240348,
       -0.13322628, -0.09946095, -0.07857978, -0.1609999 , -0.1226378 ,
       -0.16153783, -0.15507192, -0.1112509 , -0.13194976, -0.11351561,
       -0.12206051, -0.11960744, -0.14217492, -0.18590655, -0.09495189,
       -0.14684957, -0.13792519, -0.14985352, -0.10745632, -0.10344257,
       -0.1487565 , -0.10280168, -0.17277341, -0.13212576, -0.1275964 ,
       -0.11307298, -0.14628022, -0.13503592, -0.13217259, -0.14552261,
       -0.09595366, -0.09691823, -0.14599636, -0.14468743, -0.08347621,
       -0.13515473, -0.15529284, -0.17148644, -0.13425866, -0.13139252,
       -0.05913334, -0.13980873, -0.15018628, -0.17499415, -0.08525192,
       -0.15399992, -0.09146173, -0.13794207, -0.14665661, -0.1458897 ,
       -0.10867145, -0.11669592, -0.1258592 , -0.12509763, -0.09617766,
       -0.15780202, -0.13460961, -0.0864504 , -0.17140504, -0.10219992,
       -0.14913827, -0.09368758, -0.13352921, -0.10875017, -0.16

In [86]:
dwIIHHatNormF2[:, 0]

array([0.        , 0.82798562, 0.81486093, 0.8229549 , 0.81577824,
       0.82005575, 0.81906725, 0.81644441, 0.81268897, 0.8203099 ,
       0.81880115, 0.82647204, 0.82265558, 0.82521223, 0.82186896,
       0.81022157, 0.8153717 , 0.81587403, 0.81784131, 0.81119063,
       0.82237981, 0.82721961, 0.82232393, 0.82500126, 0.82602062,
       0.82541087, 0.81787708, 0.82576758, 0.82136467, 0.81747683,
       0.82523512, 0.82405776, 0.82033886, 0.82226342, 0.82241619,
       0.81863983, 0.81628076, 0.81421187, 0.81947473, 0.82053564,
       0.82131966, 0.82319369, 0.82844075, 0.829315  , 0.82378095,
       0.81716834, 0.81679567, 0.82346739, 0.82112847, 0.82183348,
       0.81889169, 0.82406459, 0.81657239, 0.82169609, 0.82030284,
       0.8197252 , 0.81796728, 0.82320097, 0.81792887, 0.82591159,
       0.82289618, 0.82060531, 0.82088392, 0.81700295, 0.81856198,
       0.82135215, 0.81860563, 0.81758569, 0.82188842, 0.82991731,
       0.81408861, 0.82382107, 0.8182638 , 0.82234697, 0.82294

In [28]:
# compare the fake dwII with the real dWII

In [199]:
# plotting the full and sparse versions of dwII

vlims = (-1, 1)

fig7, ax7 = plt.subplots(2, 2, figsize=(17, 8))
ax7 = ax7.ravel()
weight_matrix(ax7[0], dwIIHHatNorm, xlabel='Post Index', ylabel='Pre Index',
              clabel='Weight (pA)', limsMethod='custom', vlims=vlims)
weight_matrix(ax7[1], dwIIHHatNormF, xlabel='Post Index', ylabel='Pre Index',
              clabel='Weight (pA)', limsMethod='custom', vlims=vlims)
weight_matrix(ax7[2], dwIIHHatNormF2, xlabel='Post Index', ylabel='Pre Index',
              clabel='Weight (pA)', limsMethod='custom', vlims=vlims)
weight_matrix(ax7[3], dwIIHHatNormF3, xlabel='Post Index', ylabel='Pre Index',
              clabel='Weight (pA)', limsMethod='custom', vlims=vlims)

In [97]:
dwIIHHatNorm.mean()

0.015882438916602622

In [98]:
dwIIHHatNormF.mean()

0.01413995413297265

In [99]:
dwIIHHatNormF2.mean()

0.3873928300862871

In [10]:
nTrials = R1.trialUpFRExc.size

dwIIHDiagMeanTrial = np.empty((nTrials))
dwIIHOffDiagMeanTrial = np.empty((nTrials))

for trialInd in range(nTrials):
    movAvgUpFRInhUnits = R1.trialUpFRInhUnits[trialInd, :] * Hz
    dwIIH = -np.outer(movAvgUpFRInhUnits, (p['setUpFRInh'] - movAvgUpFRInhUnits))
    dwIIHDiagMeanTrial[trialInd] = np.diagonal(dwIIH).mean()
    dwIIHOffDiagMeanTrial[trialInd] = dwIIH[~np.eye(dwIIH.shape[0],dtype=bool)].mean()

In [39]:
(dwIIHOffDiagMeanTrial.mean() - dwIIHDiagMeanTrial.mean()) / (dwIIHSTrial[startInd:endInd].mean() - dwIIHSOffTrial[startInd:endInd].mean())

30.87715328250213

In [26]:
nTrials = R1.trialUpFRExc.size
nSynapses = R1.preII.size

dwIIHSTrial = np.empty((nTrials, nSynapses))
dwIIHSOffTrial = np.empty((nTrials, nSynapses * 3))

for trialInd in range(nTrials):
    movAvgUpFRInhUnits = R1.trialUpFRInhUnits[trialInd, :] * Hz
    dwIIH = -np.outer(movAvgUpFRInhUnits, (p['setUpFRInh'] - movAvgUpFRInhUnits))
    dwIIHSTrial[trialInd, :] = dwIIH[R1.preII, R1.posII]
    dwIIH[R1.preII, R1.posII] = np.nan
    dwIIHSOffTrial[trialInd, :] = dwIIH[~np.isnan(dwIIH)]

In [16]:
R1.wEE_init.shape

(640000,)

In [149]:
dwIIHHat = dwIIHTrial.mean(2)
dwIIHSHat = np.nanmean(dwIIHSTrial, 2)

del dwIIHTrial
del dwIIHSTrial

In [145]:
dwIIHSTrial.shape

(6765, 40000)

In [27]:
startInd = 400
endInd = R1.trialUpFRExc.size

In [28]:
dwIIHSTrial[startInd:endInd].mean(1) - dwIIHSOffTrial[startInd:endInd].mean(1)

array([-0.54487334, -0.42356196, -0.27486952, ..., -0.53814998,
       -1.16753178,  0.16713672])

In [29]:
dwIIHSTrial[startInd:endInd].mean() - dwIIHSOffTrial[startInd:endInd].mean()

-0.3729681328644472

In [14]:
actualHuge = dwIIHSTrial[startInd:endInd, :].ravel()
unusedHuge = dwIIHSOffTrial[startInd:endInd, :].ravel()

In [15]:
useMin = np.min([np.min(actualHuge), np.min(unusedHuge),])
useMax = np.max([np.max(actualHuge), np.max(unusedHuge),])

In [17]:
useBins = np.linspace(useMin, useMax, 1000)

In [18]:
f, ax = plt.subplots()
ax.hist(actualHuge, bins=useBins, density=True, histtype='step')
ax.hist(unusedHuge, bins=useBins, density=True, histtype='step')
ax.legend(('actual', 'all'))

In [ ]:
x = actualHuge
x_d = np.linspace(useMin, useMax, 100)

# instantiate and fit the KDE model
kde = KernelDensity(bandwidth=0.1, kernel='gaussian')
kde.fit(x[:, None])

# score_samples returns the log of the probability density
logprob = kde.score_samples(x_d[:, None])

f, ax = plt.subplots()
ax.fill_between(x_d, np.exp(logprob), alpha=0.5)

In [ ]:
test = 0

In [213]:

mixUpFRs = False
normByIncomingConns = False
normByOutgoingConns = False
normByBothCH = False
normByBothH = False
normIncomingH = False
normHCheat = False
convergentInd = 400
deviation = 0.2
# E1 = R1.trialUpFRExc[convergentInd]
# E2 = R2.trialUpFRExc[convergentInd]
E = 6  # 4 or 6
I = 16.8  # 16.8 or 11.2
useIndex = 402

# movAvgUpFRExcUnits = np.ones((R1.p['nExc'])) * E * Hz
# movAvgUpFRInhUnits = np.ones((R1.p['nInh'])) * I * Hz

# movAvgUpFRExcUnits = np.linspace(p['setUpFRExc'] * 0.8, p['setUpFRExc'] * 1.2,  R1.p['nExc']) + 0.2 * p['setUpFRExc']
# movAvgUpFRInhUnits = np.linspace(p['setUpFRInh'] * 0.8, p['setUpFRInh'] * 1.2,  R1.p['nInh']) + 0.2 * p['setUpFRInh']

aEE = adjacency_matrix_from_flat_inds(p['nExc'], p['nExc'], R1.preEE, R1.posEE)
aIE = adjacency_matrix_from_flat_inds(p['nExc'], p['nInh'], R1.preIE, R1.posIE)
aEI = adjacency_matrix_from_flat_inds(p['nInh'], p['nExc'], R1.preEI, R1.posEI)
aII = adjacency_matrix_from_flat_inds(p['nInh'], p['nInh'], R1.preII, R1.posII)

nOutgoingToExcFromEachExc = aEE.sum(1)
nOutgoingToExcFromEachInh = aEI.sum(1)
nOutgoingToInhFromEachExc = aIE.sum(1)
nOutgoingToInhFromEachInh = aII.sum(1)

nIncomingExcOntoEachExc = aEE.sum(0)
nIncomingInhOntoEachExc = aEI.sum(0)
nIncomingExcOntoEachInh = aIE.sum(0)
nIncomingInhOntoEachInh = aII.sum(0)

if mixUpFRs:
    movAvgUpFRExcUnits[:int(R1.p['nExc'] / 3)] = p['setUpFRExc'] * (1 - deviation)
    movAvgUpFRExcUnits[int(R1.p['nExc'] / 3):] = p['setUpFRExc'] * (1 + deviation)
    movAvgUpFRInhUnits[:int(R1.p['nInh'] / 3)] = p['setUpFRInh'] * (1 - deviation)
    movAvgUpFRInhUnits[int(R1.p['nInh'] / 3):] = p['setUpFRInh'] * (1 + deviation)

movAvgUpFRExcUnits = R1.trialUpFRExcUnits[useIndex, :] * Hz
movAvgUpFRInhUnits = R1.trialUpFRInhUnits[useIndex, :] * Hz
    
# each of these is preInds elements long, so must be tiled by postInds
dwEECH1 = movAvgUpFRExcUnits * (p['setUpFRInh'] - movAvgUpFRInhUnits.mean()) / Hz / Hz
dwEICH1 = -movAvgUpFRInhUnits * (p['setUpFRInh'] - movAvgUpFRInhUnits.mean()) / Hz / Hz 
dwIECH1 = -movAvgUpFRExcUnits * (p['setUpFRExc'] - movAvgUpFRExcUnits.mean()) / Hz / Hz 
dwIICH1 = movAvgUpFRInhUnits * (p['setUpFRExc'] - movAvgUpFRExcUnits.mean()) / Hz / Hz 

# if normByOutgoingConns:
#     dwEECH1 *= (p['nExc'] * p['propConnect']) / nOutgoingToExcFromEachExc
#     dwEICH1 *= (p['nInh'] * p['propConnect']) / nOutgoingToExcFromEachInh
#     dwIECH1 *= (p['nExc'] * p['propConnect']) / nOutgoingToInhFromEachExc
#     dwIICH1 *= (p['nInh'] * p['propConnect']) / nOutgoingToInhFromEachInh

dwEECH = np.tile(dwEECH1.reshape(-1, 1), wEEInit2.shape[1])
dwEICH = np.tile(dwEICH1.reshape(-1, 1), wEIInit2.shape[1])
dwIECH = np.tile(dwIECH1.reshape(-1, 1), wIEInit2.shape[1])
dwIICH = np.tile(dwIICH1.reshape(-1, 1), wIIInit2.shape[1])

# norm by incoming AND outgoing
normMatEE = (p['nExc'] * p['nExc'] * p['propConnect'] ** 2) / np.outer(nOutgoingToExcFromEachExc, nIncomingExcOntoEachExc)
normMatEI = (p['nExc'] * p['nInh'] * p['propConnect'] ** 2) / np.outer(nOutgoingToExcFromEachInh, nIncomingInhOntoEachExc)
normMatIE = (p['nExc'] * p['nInh'] * p['propConnect'] ** 2) / np.outer(nOutgoingToInhFromEachExc, nIncomingExcOntoEachInh)
normMatII = (p['nInh'] * p['nInh'] * p['propConnect'] ** 2) / np.outer(nOutgoingToInhFromEachInh, nIncomingInhOntoEachInh) 

# norm by incoming

if normByOutgoingConns:
    dwEECH *= (p['nExc'] * p['propConnect']) / nOutgoingToExcFromEachExc.reshape(-1, 1)
    dwEICH *= (p['nInh'] * p['propConnect']) / nOutgoingToExcFromEachInh.reshape(-1, 1)
    dwIECH *= (p['nExc'] * p['propConnect']) / nOutgoingToInhFromEachExc.reshape(-1, 1)
    dwIICH *= (p['nInh'] * p['propConnect']) / nOutgoingToInhFromEachInh.reshape(-1, 1)

if normByIncomingConns:
    dwEECH *= ((p['nExc'] * p['propConnect']) / nIncomingExcOntoEachExc).reshape(1, -1)
    dwEICH *= ((p['nInh'] * p['propConnect']) / nIncomingInhOntoEachExc).reshape(1, -1)
    dwIECH *= ((p['nExc'] * p['propConnect']) / nIncomingExcOntoEachInh).reshape(1, -1)
    dwIICH *= ((p['nInh'] * p['propConnect']) / nIncomingInhOntoEachInh).reshape(1, -1)

if normByBothCH:
    dwEECH *= normMatEE
    dwEICH *= normMatEI
    dwIECH *= normMatIE
    dwIICH *= normMatII

# regular homeo (outer product)
# since outer strips units and because of alpha we multiply by Hz to convert to amps
dwEEH = np.outer(movAvgUpFRExcUnits, (p['setUpFRExc'] - movAvgUpFRExcUnits))
dwEIH = -np.outer(movAvgUpFRInhUnits, (p['setUpFRExc'] - movAvgUpFRExcUnits)) 
dwIEH = np.outer(movAvgUpFRExcUnits, (p['setUpFRInh'] - movAvgUpFRInhUnits)) 
dwIIH = -np.outer(movAvgUpFRInhUnits, (p['setUpFRInh'] - movAvgUpFRInhUnits)) 

if normByBothH:
    dwEEH *= normMatEE
    dwEIH *= normMatEI
    dwIEH *= normMatIE
    dwIIH *= normMatII

if normIncomingH:
    dwEEH *= ((p['nExc'] * p['propConnect']) / nIncomingExcOntoEachExc).reshape(1, -1)
    dwEIH *= ((p['nInh'] * p['propConnect']) / nIncomingInhOntoEachExc).reshape(1, -1)
    dwIEH *= ((p['nExc'] * p['propConnect']) / nIncomingExcOntoEachInh).reshape(1, -1)
    dwIIH *= ((p['nInh'] * p['propConnect']) / nIncomingInhOntoEachInh).reshape(1, -1)
    
if normHCheat:
    pass

dwEE = dwEECH + dwEEH
dwEI = dwEICH + dwEIH
dwIE = dwIECH + dwIEH
dwII = dwIICH + dwIIH

dwEES = np.full((R1.p['nExc'], R1.p['nExc']), np.nan, dtype=np.float32)
dwEIS = np.full((R1.p['nInh'], R1.p['nExc']), np.nan, dtype=np.float32)
dwIES = np.full((R1.p['nExc'], R1.p['nInh']), np.nan, dtype=np.float32)
dwIIS = np.full((R1.p['nInh'], R1.p['nInh']), np.nan, dtype=np.float32)

dwEES[R1.preEE, R1.posEE] = dwEE[R1.preEE, R1.posEE]
dwEIS[R1.preEI, R1.posEI] = dwEI[R1.preEI, R1.posEI]
dwIES[R1.preIE, R1.posIE] = dwIE[R1.preIE, R1.posIE]
dwIIS[R1.preII, R1.posII] = dwII[R1.preII, R1.posII]

dwEECHS = np.full((R1.p['nExc'], R1.p['nExc']), np.nan, dtype=np.float32)
dwEICHS = np.full((R1.p['nInh'], R1.p['nExc']), np.nan, dtype=np.float32)
dwIECHS = np.full((R1.p['nExc'], R1.p['nInh']), np.nan, dtype=np.float32)
dwIICHS = np.full((R1.p['nInh'], R1.p['nInh']), np.nan, dtype=np.float32)

dwEECHS[R1.preEE, R1.posEE] = dwEECH[R1.preEE, R1.posEE]
dwEICHS[R1.preEI, R1.posEI] = dwEICH[R1.preEI, R1.posEI]
dwIECHS[R1.preIE, R1.posIE] = dwIECH[R1.preIE, R1.posIE]
dwIICHS[R1.preII, R1.posII] = dwIICH[R1.preII, R1.posII]

dwEEHS = np.full((R1.p['nExc'], R1.p['nExc']), np.nan, dtype=np.float32)
dwEIHS = np.full((R1.p['nInh'], R1.p['nExc']), np.nan, dtype=np.float32)
dwIEHS = np.full((R1.p['nExc'], R1.p['nInh']), np.nan, dtype=np.float32)
dwIIHS = np.full((R1.p['nInh'], R1.p['nInh']), np.nan, dtype=np.float32)

dwEEHS[R1.preEE, R1.posEE] = dwEEH[R1.preEE, R1.posEE]
dwEIHS[R1.preEI, R1.posEI] = dwEIH[R1.preEI, R1.posEI]
dwIEHS[R1.preIE, R1.posIE] = dwIEH[R1.preIE, R1.posIE]
dwIIHS[R1.preII, R1.posII] = dwIIH[R1.preII, R1.posII]

In [120]:
dwIIHS[R1.preII, R1.posII] = dwIIH[R1.preII, R1.posII]

In [121]:
dwIIHS = np.full((R1.p['nInh'], R1.p['nInh']), np.nan, dtype=np.float32)
dwIIHS[R1.preII, R1.posII] = dwIIH[R1.preII, R1.posII]

In [122]:
dwIICH1 = movAvgUpFRInhUnits * (p['setUpFRExc'] - movAvgUpFRExcUnits.mean()) / Hz / Hz

In [123]:
dwIICH1

array([0.44232537, 0.44232537, 0.57242106, 0.57242106, 0.31222967,
       0.33824881, 0.54640192, 0.33824881, 0.7285359 , 0.31222967,
       0.52038279, 0.54640192, 0.39028709, 0.67649762, 0.54640192,
       0.54640192, 0.44232537, 0.52038279, 0.67649762, 0.41630623,
       0.44232537, 0.41630623, 0.5984402 , 0.23417225, 0.41630623,
       0.57242106, 0.36426795, 0.39028709, 0.52038279, 0.44232537,
       0.39028709, 0.49436365, 0.39028709, 0.52038279, 0.44232537,
       0.54640192, 0.44232537, 0.52038279, 0.67649762, 0.49436365,
       0.49436365, 0.65047848, 0.88465074, 0.46834451, 0.52038279,
       0.36426795, 0.44232537, 0.36426795, 0.36426795, 0.7285359 ,
       0.41630623, 0.44232537, 0.28621053, 0.46834451, 0.67649762,
       0.49436365, 0.49436365, 0.33824881, 0.57242106, 0.41630623,
       0.54640192, 0.28621053, 0.57242106, 0.54640192, 0.75455504,
       0.46834451, 0.39028709, 0.39028709, 0.57242106, 0.52038279,
       0.57242106, 0.54640192, 0.57242106, 0.67649762, 0.54640

In [124]:
dwIIH = -np.outer(movAvgUpFRInhUnits, (p['setUpFRInh'] - movAvgUpFRInhUnits)) 

In [125]:
dwIIH

array([[-18.75      , -18.75      ,  27.20588235, ...,  -0.36764706,
        -37.13235294, -55.51470588],
       [-18.75      , -18.75      ,  27.20588235, ...,  -0.36764706,
        -37.13235294, -55.51470588],
       [-24.26470588, -24.26470588,  35.20761246, ...,  -0.47577855,
        -48.05363322, -71.84256055],
       ...,
       [-20.95588235, -20.95588235,  30.40657439, ...,  -0.41089965,
        -41.50086505, -62.04584775],
       [-16.54411765, -16.54411765,  24.00519031, ...,  -0.32439446,
        -32.76384083, -48.98356401],
       [-14.33823529, -14.33823529,  20.80449827, ...,  -0.28114187,
        -28.39532872, -42.45242215]])

In [126]:
movAvgUpFRInhUnits[0]

12.5 * hertz

In [127]:
(p['setUpFRInh'] - movAvgUpFRInhUnits)[0]

1.5 * hertz

In [128]:
dwIIH

array([[-18.75      , -18.75      ,  27.20588235, ...,  -0.36764706,
        -37.13235294, -55.51470588],
       [-18.75      , -18.75      ,  27.20588235, ...,  -0.36764706,
        -37.13235294, -55.51470588],
       [-24.26470588, -24.26470588,  35.20761246, ...,  -0.47577855,
        -48.05363322, -71.84256055],
       ...,
       [-20.95588235, -20.95588235,  30.40657439, ...,  -0.41089965,
        -41.50086505, -62.04584775],
       [-16.54411765, -16.54411765,  24.00519031, ...,  -0.32439446,
        -32.76384083, -48.98356401],
       [-14.33823529, -14.33823529,  20.80449827, ...,  -0.28114187,
        -28.39532872, -42.45242215]])

In [129]:
dwIIH

array([[-18.75      , -18.75      ,  27.20588235, ...,  -0.36764706,
        -37.13235294, -55.51470588],
       [-18.75      , -18.75      ,  27.20588235, ...,  -0.36764706,
        -37.13235294, -55.51470588],
       [-24.26470588, -24.26470588,  35.20761246, ...,  -0.47577855,
        -48.05363322, -71.84256055],
       ...,
       [-20.95588235, -20.95588235,  30.40657439, ...,  -0.41089965,
        -41.50086505, -62.04584775],
       [-16.54411765, -16.54411765,  24.00519031, ...,  -0.32439446,
        -32.76384083, -48.98356401],
       [-14.33823529, -14.33823529,  20.80449827, ...,  -0.28114187,
        -28.39532872, -42.45242215]])

In [130]:
print(dwIICH.mean())
print(dwIIH.mean())
print(dwII.mean())
print(np.nanmean(dwIICHS))
print(np.nanmean(dwIIHS))
print(np.nanmean(dwIIS))
print(np.nanmean(dwIIS) - dwII.mean())
print(np.nanmean(dwIIHS) - dwIIH.mean())

0.4915015408736994
-1.5319528546712946
-1.040451313797595
0.49150088
-1.7381049
-1.2466041
-0.20615277117097186
-0.2061520847894598


In [183]:
print(dwIIHHat.mean())
print(np.nanmean(dwIIHSHat))
print(np.nanmean(dwIIHSHat) - dwIIHHat.mean())

0.2517823264386186
-0.04654048658378834
-0.29832281302240693


In [159]:
dwIIH = dwIIHHat.copy()
dwIIHS = dwIIHSHat.copy()

In [156]:
dwIIHFlat = dwIIH.ravel()

In [157]:
dividerNums = np.random.rand(*dwIIHFlat.shape) * 4

dwIIHFlatSub1 = dwIIHFlat[np.logical_and(dividerNums >= 0, dividerNums < 1)]
dwIIHFlatSub2 = dwIIHFlat[np.logical_and(dividerNums >= 1, dividerNums < 2)]
dwIIHFlatSub3 = dwIIHFlat[np.logical_and(dividerNums >= 2, dividerNums < 3)]
dwIIHFlatSub4 = dwIIHFlat[np.logical_and(dividerNums >= 3, dividerNums < 4)]

diffSub1 = dwIIHFlatSub1.mean() - dwIIHFlat.mean()
diffSub2 = dwIIHFlatSub2.mean() - dwIIHFlat.mean()
diffSub3 = dwIIHFlatSub3.mean() - dwIIHFlat.mean()
diffSub4 = dwIIHFlatSub4.mean() - dwIIHFlat.mean()

print(diffSub1)
print(diffSub2)
print(diffSub3)
print(diffSub4)
print((diffSub1 + diffSub2 + diffSub3 + diffSub4) / 4)

0.0017841540850144222
-0.0036508644799483625
0.0029813964254712944
-0.0011026936302766521
2.9981000651754797e-06


In [158]:
nSamples = 1000
diffArray = np.empty((nSamples,))
for sampleInd in range(nSamples):
    diffArray[sampleInd] = np.random.choice(dwIIHFlat, int(dwIIHFlat.size * p['propConnect']), replace=False).mean() - dwIIH.mean()

print(diffArray.mean())

8.919040712190734e-05


In [141]:
dwEIHFlat = dwEIH.ravel()

nSamples = 1000
diffArray = np.empty((nSamples,))
for sampleInd in range(nSamples):
    diffArray[sampleInd] = np.random.choice(dwEIHFlat, int(dwEIHFlat.size * p['propConnect']), replace=False).mean() - dwEIH.mean()

print(diffArray.mean())

0.0027594642084774437


In [142]:
dwII.shape

(400, 400)

In [160]:
# can't pick the diagonal...

probabilityArray = np.full((dwII.shape[0], dwII.shape[-1]), 1 / (dwII.shape[0] * (dwII.shape[-1] - 1)))
probabilityArray[np.diag_indices_from(probabilityArray)] = 0

nSamples = 1000
diffArray = np.empty((nSamples,))
for sampleInd in range(nSamples):
    currentRandomSubset = np.random.choice(dwIIHFlat, int(dwIIHFlat.size * p['propConnect']), replace=False, p=probabilityArray.ravel())
    diffArray[sampleInd] = currentRandomSubset.mean() - dwIIH.mean()

print(diffArray.mean())

-0.028636649206147724


In [174]:
# check the diagonals

diagonalInds = np.arange(-dwII.shape[0], dwII.shape[0])
diagonalMeans = np.zeros(*diagonalInds.shape)
diagonalMeansS = np.zeros(*diagonalInds.shape)

for diagonalInd, diagonalVal in enumerate(diagonalInds):
    diagonalMeans[diagonalInd] = np.diagonal(dwIIH, offset=diagonalVal).mean()
    diagonalMeansS[diagonalInd] = np.nanmean(np.diagonal(dwIIHS, offset=diagonalVal))

WARNING    C:\Users\mikejseay\anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: Mean of empty slice.
  
 [py.warnings]
WARNING    C:\Users\mikejseay\anaconda3\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
 [py.warnings]
WARNING    C:\Users\mikejseay\anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: Mean of empty slice
  if __name__ == '__main__':
 [py.warnings]


In [175]:
f, ax = plt.subplots()
ax.plot(diagonalInds, diagonalMeansS)
ax.plot(diagonalInds, diagonalMeans)
ax.legend(('actual', 'full'))

In [176]:
np.nanmean(diagonalMeansS) - np.nanmean(diagonalMeans)

-0.285683387497191

In [177]:
dwIIHS.shape

(400, 400)

In [178]:
# check the rows / columns

rowMeans = dwIIH.mean(0)
rowMeansS = np.nanmean(dwIIHS, 0)

f, ax = plt.subplots()
ax.plot(np.arange(dwIIH.shape[0]), rowMeansS)
ax.plot(np.arange(dwIIH.shape[0]), rowMeans)
ax.legend(('actual', 'full'))

In [179]:
rowMeansS.mean() - rowMeans.mean()

-0.2901399698088718

In [180]:
rowMeansS.mean()

-0.03835764337025324

In [181]:
rowMeans.mean()

0.2517823264386186

In [182]:
# check the rows / columns

colMeans = dwIIH.mean(1)
colMeansS = np.nanmean(dwIIHS, 1)

f, ax = plt.subplots()
ax.plot(np.arange(dwIIH.shape[0]), colMeansS)
ax.plot(np.arange(dwIIH.shape[0]), colMeans)
ax.legend(('actual', 'full'))

In [172]:
colMeansS.mean() - colMeans.mean()

-0.2994400631439752

In [63]:
probabilityArray.ravel()

array([0.00000000e+00, 6.26566416e-06, 6.26566416e-06, ...,
       6.26566416e-06, 6.26566416e-06, 0.00000000e+00])

In [64]:
probabilityArray.sum()

1.0000000000000002

In [65]:
int(dwIIHFlat.size * p['propConnect'])

40000

In [66]:
dwIIHFlat.shape

(160000,)

In [ ]:
dwIIRand = 

In [67]:
print(dwEECH.mean())
print(dwEEH.mean())
print(dwEE.mean())
print(np.nanmean(dwEECHS))
print(np.nanmean(dwEEHS))
print(np.nanmean(dwEES))
print(np.nanmean(dwEES) - dwEE.mean())

0.7014109469615014
0.22089890212748914
0.92230984908899
0.7014309
0.19239311
0.8938241
-0.028485748594605287


In [68]:
print(dwEICH.mean())
print(dwEIH.mean())
print(dwEI.mean())
print(np.nanmean(dwEICHS))
print(np.nanmean(dwEIHS))
print(np.nanmean(dwEIS))
print(np.nanmean(dwEIS) - dwEI.mean())

-1.961582909818328
-0.6177712410791583
-2.579354150897485
-1.9631215
-0.70788276
-2.6710038
-0.09164966761447779


In [69]:
print(dwIECH.mean())
print(dwIEH.mean())
print(dwIE.mean())
print(np.nanmean(dwIECHS))
print(np.nanmean(dwIEHS))
print(np.nanmean(dwIES))
print(np.nanmean(dwIES) - dwIE.mean())

-0.22089890212748856
0.7014109469615104
0.4805120448340219
-0.221114
0.6341471
0.41303325
-0.06747879784000332


In [173]:
# plotting the full and sparse versions of dwII

allNumbers = np.concatenate((dwII, dwIIS)).ravel()

# vlims = (np.nanmin(allNumbers), np.nanmax(allNumbers))
vlims = (-np.nanmax(np.fabs(allNumbers)), np.nanmax(np.fabs(allNumbers)))

fig7, ax7 = plt.subplots(1, 2, figsize=(17, 8))
weight_matrix(ax7[0], dwII, xlabel='Post Index', ylabel='Pre Index',
              clabel='Weight (pA)', limsMethod='custom', vlims=vlims)
weight_matrix(ax7[1], dwIIS, xlabel='Post Index', ylabel='Pre Index',
              clabel='Weight (pA)', limsMethod='custom', vlims=vlims)

In [1110]:
# plotting the distribution of the weights that are kept in the sparse vs left out

In [ ]:
dwII

In [1148]:
dwEERemConn = dwEE.copy()
dwEIRemConn = dwEI.copy()
dwIERemConn = dwIE.copy()
dwIIRemConn = dwII.copy()

dwEERemConn[R1.preEE, R1.posEE] = np.nan
dwEIRemConn[R1.preEI, R1.posEI] = np.nan
dwIERemConn[R1.preIE, R1.posIE] = np.nan
dwIIRemConn[R1.preII, R1.posII] = np.nan

nBins = 100

f, ax = plt.subplots(2, 2)

ax[0, 0].hist(dwEE[R1.preEE, R1.posEE], bins=nBins, density=True, histtype='step')
ax[0, 1].hist(dwEI[R1.preEI, R1.posEI], bins=nBins, density=True, histtype='step')
ax[1, 0].hist(dwIE[R1.preIE, R1.posIE], bins=nBins, density=True, histtype='step')
ax[1, 1].hist(dwII[R1.preII, R1.posII], bins=nBins, density=True, histtype='step')

ax[0, 0].hist(dwEERemConn.ravel(), bins=nBins, density=True, histtype='step')
ax[0, 1].hist(dwEIRemConn.ravel(), bins=nBins, density=True, histtype='step')
ax[1, 0].hist(dwIERemConn.ravel(), bins=nBins, density=True, histtype='step')
ax[1, 1].hist(dwIIRemConn.ravel(), bins=nBins, density=True, histtype='step')

WARNING    C:\Users\mikejseay\anaconda3\lib\site-packages\numpy\lib\histograms.py:839: RuntimeWarning: invalid value encountered in greater_equal
  keep = (tmp_a >= first_edge)
 [py.warnings]
WARNING    C:\Users\mikejseay\anaconda3\lib\site-packages\numpy\lib\histograms.py:840: RuntimeWarning: invalid value encountered in less_equal
  keep &= (tmp_a <= last_edge)
 [py.warnings]
WARNING    C:\Users\mikejseay\anaconda3\lib\site-packages\numpy\lib\histograms.py:839: RuntimeWarning: invalid value encountered in greater_equal
  keep = (tmp_a >= first_edge)
 [py.warnings]
WARNING    C:\Users\mikejseay\anaconda3\lib\site-packages\numpy\lib\histograms.py:840: RuntimeWarning: invalid value encountered in less_equal
  keep &= (tmp_a <= last_edge)
 [py.warnings]
WARNING    C:\Users\mikejseay\anaconda3\lib\site-packages\numpy\lib\histograms.py:839: RuntimeWarning: invalid value encountered in greater_equal
  keep = (tmp_a >= first_edge)
 [py.warnings]
WARNING    C:\Users\mikejseay\anaconda3\lib\si

(array([1.65828622e-06, 1.65828622e-06, 3.31657245e-06, 1.65828622e-06,
        6.63314489e-06, 1.32662898e-05, 4.97485867e-06, 8.29143112e-06,
        2.32160071e-05, 3.15074382e-05, 4.14571556e-05, 1.32662898e-05,
        6.63314489e-05, 8.78891698e-05, 1.27688039e-04, 7.79394525e-05,
        1.39296043e-04, 1.98994347e-04, 2.86883517e-04, 3.68139542e-04,
        4.36129277e-04, 5.90349896e-04, 7.71103094e-04, 9.08740850e-04,
        1.24868953e-03, 1.51069875e-03, 2.12426465e-03, 2.16074695e-03,
        2.85059402e-03, 3.18888441e-03, 4.39280021e-03, 4.65149286e-03,
        5.45741996e-03, 6.23018134e-03, 6.68621005e-03, 6.97475186e-03,
        7.11736447e-03, 7.08585703e-03, 7.19032906e-03, 6.01626242e-03,
        1.02150431e-02, 6.75254150e-03, 1.62611547e-02, 4.91018551e-03,
        5.98143841e-03, 7.80555325e-03, 6.88354611e-03, 8.52856605e-03,
        7.23841936e-03, 6.17711618e-03, 5.59505772e-03, 4.73109060e-03,
        4.21038872e-03, 3.45586849e-03, 2.81411172e-03, 2.799187

In [1173]:
dwIIHSRemConn = dwIIH.copy()
dwIIHSRemConn[R1.preII, R1.posII] = np.nan

useBins = np.linspace(dwIIH.min(), dwIIH.max(), 100)

f, ax = plt.subplots()
ax.hist(dwIIHS[R1.preII, R1.posII], bins=useBins, density=True, histtype='step')
ax.hist(dwIIHSRemConn.ravel(), bins=useBins, density=True, histtype='step')
ax.legend(('actual', 'all'))

In [1164]:
dwIICHSRemConn = dwIICH.copy()
dwIICHSRemConn[R1.preII, R1.posII] = np.nan

useBins = np.linspace(dwIICH.min(), dwIICH.max(), 100)

f, ax = plt.subplots()
ax.hist(dwIICHS[R1.preII, R1.posII], bins=useBins, density=True, histtype='step')
ax.hist(dwIICHSRemConn.ravel(), bins=useBins, density=True, histtype='step')
ax.legend(('actual', 'all'))

In [1104]:
np.nanmean(dwIIS) - dwII.mean()

-0.17975236959979113

In [1060]:
plt.close('all')

In [985]:
# plotting the cross-homeo and homeo dW matrices next to each other

dwFullCH = np.block([[dwEECH, dwIECH], [-dwEICH, -dwIICH]])
dwFullH = np.block([[dwEEH, dwIEH], [-dwEIH, -dwIIH]])

allNumbers = np.concatenate((dwFullCH, dwFullH)).ravel()

# vlims = (np.nanmin(allNumbers), np.nanmax(allNumbers))
vlims = (-np.nanmax(np.fabs(allNumbers)), np.nanmax(np.fabs(allNumbers)))

wFullInitPlot = dwFullCH.copy()
wFullFinalPlot = dwFullH.copy()

fig7, ax7 = plt.subplots(1, 2, figsize=(17, 8))
weight_matrix(ax7[0], wFullInitPlot, xlabel='Post Index', ylabel='Pre Index',
              clabel='Weight (pA)', limsMethod='custom', vlims=vlims)
weight_matrix(ax7[1], wFullFinalPlot, xlabel='Post Index', ylabel='Pre Index',
              clabel='Weight (pA)', limsMethod='custom', vlims=vlims)

In [986]:
# plotting the two-term sum compared to its sparse version

dwFull = np.block([[dwEE, dwIE], [-dwEI, -dwII]])
dwFullS = np.block([[dwEES, dwIES], [-dwEIS, -dwIIS]])

allNumbers = np.concatenate((dwFull, dwFullS)).ravel()

# vlims = (np.nanmin(allNumbers), np.nanmax(allNumbers))
vlims = (-np.nanmax(np.fabs(allNumbers)), np.nanmax(np.fabs(allNumbers)))

wFullInitPlot = dwFull.copy()
wFullFinalPlot = dwFullS.copy()

wFullInitPlot[np.isnan(wFullInitPlot)] = 0
wFullFinalPlot[np.isnan(wFullFinalPlot)] = 0

fig7, ax7 = plt.subplots(1, 2, figsize=(17, 8))
weight_matrix(ax7[0], wFullInitPlot, xlabel='Post Index', ylabel='Pre Index',
              clabel='Weight (pA)', limsMethod='custom', vlims=vlims)
weight_matrix(ax7[1], wFullFinalPlot, xlabel='Post Index', ylabel='Pre Index',
              clabel='Weight (pA)', limsMethod='custom', vlims=vlims)

In [987]:
incomingExcCurrentOntoEachExcCH = np.nansum(dwEECHS, 0)
incomingInhCurrentOntoEachExcCH = np.nansum(dwEICHS, 0)
incomingExcCurrentOntoEachInhCH = np.nansum(dwIECHS, 0)
incomingInhCurrentOntoEachInhCH = np.nansum(dwIICHS, 0)
outgoingCurrentToExcFromEachExcCH = np.nansum(dwEECHS, 1)
outgoingCurrentToExcFromEachInhCH = np.nansum(dwEICHS, 1)
outgoingCurrentToInhFromEachExcCH = np.nansum(dwIECHS, 1)
outgoingCurrentToInhFromEachInhCH = np.nansum(dwIICHS, 1)

incomingExcCurrentOntoEachExcH = np.nansum(dwEEHS, 0)
incomingInhCurrentOntoEachExcH = np.nansum(dwEIHS, 0)
incomingExcCurrentOntoEachInhH = np.nansum(dwIEHS, 0)
incomingInhCurrentOntoEachInhH = np.nansum(dwIIHS, 0)
outgoingCurrentToExcFromEachExcH = np.nansum(dwEEHS, 1)
outgoingCurrentToExcFromEachInhH = np.nansum(dwEIHS, 1)
outgoingCurrentToInhFromEachExcH = np.nansum(dwIEHS, 1)
outgoingCurrentToInhFromEachInhH = np.nansum(dwIIHS, 1)

incomingExcCurrentOntoEachExc = np.nansum(dwEES, 0)
incomingInhCurrentOntoEachExc = np.nansum(dwEIS, 0)
incomingExcCurrentOntoEachInh = np.nansum(dwIES, 0)
incomingInhCurrentOntoEachInh = np.nansum(dwIIS, 0)
outgoingCurrentToExcFromEachExc = np.nansum(dwEES, 1)
outgoingCurrentToExcFromEachInh = np.nansum(dwEIS, 1)
outgoingCurrentToInhFromEachExc = np.nansum(dwIES, 1)
outgoingCurrentToInhFromEachInh = np.nansum(dwIIS, 1)

In [961]:
incomingExcCurrentOntoEachExcCH

array([-6713.287 , -6714.1016, -6713.2764, ..., -6704.177 , -6691.0044,
       -6704.8345], dtype=float32)

In [925]:
labels = ['incomingExcCurrentOntoEachExcCH', 'incomingInhCurrentOntoEachExcCH', 'incomingExcCurrentOntoEachInhCH', 'incomingInhCurrentOntoEachInhCH', 'outgoingCurrentToExcFromEachExcCH', 'outgoingCurrentToExcFromEachInhCH', 'outgoingCurrentToInhFromEachExcCH', 'outgoingCurrentToInhFromEachInhCH', 'incomingExcCurrentOntoEachExcH', 'incomingInhCurrentOntoEachExcH', 'incomingExcCurrentOntoEachInhH', 'incomingInhCurrentOntoEachInhH', 'outgoingCurrentToExcFromEachExcH', 'outgoingCurrentToExcFromEachInhH', 'outgoingCurrentToInhFromEachExcH', 'outgoingCurrentToInhFromEachInhH', 'incomingExcCurrentOntoEachExc', 'incomingInhCurrentOntoEachExc', 'incomingExcCurrentOntoEachInh', 'incomingInhCurrentOntoEachInh', 'outgoingCurrentToExcFromEachExc', 'outgoingCurrentToExcFromEachInh', 'outgoingCurrentToInhFromEachExc', 'outgoingCurrentToInhFromEachInh',]

In [926]:
allCVs = np.array([cv(incomingExcCurrentOntoEachExcCH), cv(incomingInhCurrentOntoEachExcCH), cv(incomingExcCurrentOntoEachInhCH), cv(incomingInhCurrentOntoEachInhCH), cv(outgoingCurrentToExcFromEachExcCH), cv(outgoingCurrentToExcFromEachInhCH), cv(outgoingCurrentToInhFromEachExcCH), cv(outgoingCurrentToInhFromEachInhCH), cv(incomingExcCurrentOntoEachExcH), cv(incomingInhCurrentOntoEachExcH), cv(incomingExcCurrentOntoEachInhH), cv(incomingInhCurrentOntoEachInhH), cv(outgoingCurrentToExcFromEachExcH), cv(outgoingCurrentToExcFromEachInhH), cv(outgoingCurrentToInhFromEachExcH), cv(outgoingCurrentToInhFromEachInhH), cv(incomingExcCurrentOntoEachExc), cv(incomingInhCurrentOntoEachExc), cv(incomingExcCurrentOntoEachInh), cv(incomingInhCurrentOntoEachInh), cv(outgoingCurrentToExcFromEachExc), cv(outgoingCurrentToExcFromEachInh), cv(outgoingCurrentToInhFromEachExc), cv(outgoingCurrentToInhFromEachInh), ])

In [927]:
cvDF = pd.DataFrame(allCVs * 100, labels)

In [928]:
cvDF

,0
incomingExcCurrentOntoEachExcCH,-0.418812
incomingInhCurrentOntoEachExcCH,0.837739
incomingExcCurrentOntoEachInhCH,0.423381
incomingInhCurrentOntoEachInhCH,-0.861423
outgoingCurrentToExcFromEachExcCH,-10.440950
outgoingCurrentToExcFromEachInhCH,10.383132
outgoingCurrentToInhFromEachExcCH,13.013013
outgoingCurrentToInhFromEachInhCH,-12.788823
incomingExcCurrentOntoEachExcH,-57.980984
incomingInhCurrentOntoEachExcH,58.653854


In [812]:
allCVs

array([-0.04245139,  0.08597401,  0.04286776, -0.08697999, -0.10442287,
        0.1038627 ,  0.13032407, -0.12737298, -0.57980984,  0.58653855,
       -0.5805523 ,  0.5879602 , -0.10687449,  0.10649461, -0.13842519,
        0.13689326, -0.15769655,  0.1769976 , -0.902717  ,  0.906475  ,
       -0.10449539,  0.10396127, -0.14992274,  0.1496405 ], dtype=float32)

In [528]:
print(incomingExcCurrentOntoEachExc.shape)

(1600,)


In [523]:
incomingExcCurrentOntoEachExc.std()

12.557544

In [524]:
outgoingExcCurrentToEachExc = np.nansum(dwEECHS, 1)
outgoingExcCurrentToEachExc

array([-6744.02  , -6709.878 , -6700.067 , ..., -6737.6187, -6722.0674,
       -6714.1416], dtype=float32)

In [525]:
outgoingExcCurrentToEachExc.std()

12.510517

In [526]:
np.nansum(dwEES, 0)

array([-9057.689, -8949.369, -9204.297, ..., -9171.056, -8984.055,
       -9165.846], dtype=float32)

In [527]:
np.nansum(dwEE, 0) / p['nExc']

array([-23.21856026, -24.24514108, -22.30134881, ..., -22.42062698,
       -23.67049097, -22.34091519])

In [224]:
np.nansum(dwEES, 0) / (~np.isnan(dwEES)).sum(0)

array([-22.79990959, -22.79991584, -22.799904  , ..., -22.79990473,
       -22.79991234, -22.79990424])

In [225]:
np.nansum(dwEES, 0) / (p['nExc'] * p['propConnect'])

array([-22.286911, -21.032923, -23.540901, ..., -23.369902, -21.716917,
       -23.483902], dtype=float32)

In [10]:
downSampleFactorForE = 0.025
downSampleFactorForI = 0.1

downSampleNForE = int(downSampleFactorForE * R1.p['nExc'])
downSampleNForI = int(downSampleFactorForI * R1.p['nInh'])

downSampleE = np.random.choice(R1.p['nExc'], size=downSampleNForE, replace=False)
downSampleI = np.random.choice(R1.p['nInh'], size=downSampleNForI, replace=False)

In [13]:
np.all(R1.trialUpFRExc == R2.trialUpFRExc)

True

In [14]:
np.all(R1.trialwEE == R2.trialwEE)

True

In [20]:
np.allclose(wFullInit1, wFullInit2, equal_nan=True)

True

In [21]:
np.allclose(wFullFinal1, wFullFinal2, equal_nan=True)

True

In [813]:
labels = ['incomingExcCurrentOntoEachExcCH',
'incomingInhCurrentOntoEachExcCH',
'incomingExcCurrentOntoEachInhCH',
'incomingInhCurrentOntoEachInhCH',
'outgoingCurrentToExcFromEachExcCH',
'outgoingCurrentToExcFromEachInhCH',
'outgoingCurrentToInhFromEachExcCH',
'outgoingCurrentToInhFromEachInhCH',
'incomingExcCurrentOntoEachExcH',
'incomingInhCurrentOntoEachExcH',
'incomingExcCurrentOntoEachInhH',
'incomingInhCurrentOntoEachInhH',
'outgoingCurrentToExcFromEachExcH',
'outgoingCurrentToExcFromEachInhH',
'outgoingCurrentToInhFromEachExcH',
'outgoingCurrentToInhFromEachInhH',
'incomingExcCurrentOntoEachExc',
'incomingInhCurrentOntoEachExc',
'incomingExcCurrentOntoEachInh',
'incomingInhCurrentOntoEachInh',
'outgoingCurrentToExcFromEachExc',
'outgoingCurrentToExcFromEachInh',
'outgoingCurrentToInhFromEachExc',
'outgoingCurrentToInhFromEachInh',]

In [810]:
incomingExcCurrentOntoEachExcCH
incomingInhCurrentOntoEachExcCH
incomingExcCurrentOntoEachInhCH
incomingInhCurrentOntoEachInhCH
outgoingCurrentToExcFromEachExcCH
outgoingCurrentToExcFromEachInhCH
outgoingCurrentToInhFromEachExcCH
outgoingCurrentToInhFromEachInhCH
incomingExcCurrentOntoEachExcH
incomingInhCurrentOntoEachExcH
incomingExcCurrentOntoEachInhH
incomingInhCurrentOntoEachInhH
outgoingCurrentToExcFromEachExcH
outgoingCurrentToExcFromEachInhH
outgoingCurrentToInhFromEachExcH
outgoingCurrentToInhFromEachInhH
incomingExcCurrentOntoEachExc
incomingInhCurrentOntoEachExc
incomingExcCurrentOntoEachInh
incomingInhCurrentOntoEachInh
outgoingCurrentToExcFromEachExc
outgoingCurrentToExcFromEachInh
outgoingCurrentToInhFromEachExc
outgoingCurrentToInhFromEachInh

-0.042451385
0.08597401
0.042867765
-0.086979985
-0.10442287
0.1038627
0.13032407
-0.12737298
-0.57980984
0.58653855
-0.5805523
0.5879602
-0.10687449
0.10649461
-0.13842519
0.13689326
-0.15769655
0.1769976
-0.902717
0.906475
-0.10449539
0.103961274
-0.14992274
0.1496405
